In [119]:


import scipy
from sympy import Function, Symbol, Derivative, diff, integrate, Piecewise
from sympy import sin, pi, sqrt, oo
from sympy.physics.quantum.constants import hbar
from sympy.physics.quantum.operator import DifferentialOperator, Operator
from sympy.physics.quantum.state import Wavefunction
from sympy.physics.quantum.qapply import qapply

import numpy as np
from matplotlib import pyplot as plt


In [120]:

# Define number of basis
nbasis = 10

# Construct the basis (eigenfuncitons)
x = Symbol('x')
n = Symbol('n')
m = Symbol('m')
ma = hbar/2.0
L = 20.0

g = Piecewise((0, x < 0), (0, x > L), (sqrt(2./L)*sin(n*pi*x/L), True))
phin = Wavefunction(g, x)
phim = phi.subs(n, m)

f = Function('f')
# T = -hbar/(2*ma) * DifferentialOperator(diff(f(x), x, x), f(x))
T = DifferentialOperator(-hbar/(2*ma) * Derivative(f(x), x, x), f(x))
V = 1./4. * (x - L/2.)**2

In [129]:
H = T + V

25.0*(0.1*x - 1)**2 + DifferentialOperator(-1.0*Derivative(f(x), (x, 2)),f(x))

In [133]:
H.

(25.0*(0.1*x - 1)**2 + DifferentialOperator(-1.0*Derivative(f(x), (x, 2)),f(x)),
 1)

In [106]:
hmn = (phim.expr*(V * phin.expr + qapply(T * phin).expr)).simplify()
hmn

(0.00025*pi**2*n**2 + 2.5*(0.1*x - 1)**2)*sin(0.05*pi*m*x)*sin(0.05*pi*n*x)

In [102]:
# Define symbols
from sympy import symbols
y, a, b = symbols('y a b')

# Define the function to integrate
g = y**2 * sin(a*y)

In [104]:
print(type(g))
integrate(g, y)

<class 'sympy.core.mul.Mul'>


Piecewise((-y**2*cos(a*y)/a + 2*y*sin(a*y)/a**2 + 2*cos(a*y)/a**3, Ne(a, 0)), (0, True))

In [107]:
hmn.evalf(subs={m: 3, n: 3, x: L/2.0})

0.0222066099024511

In [101]:
type(hmn)

sympy.core.mul.Mul

In [88]:
integrate(hmn.expand(), (x, -oo, oo))

KeyboardInterrupt: 

In [105]:
Hmn = integrate(hmn, (x, -oo, oo))

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7fdaf73ea890>>
Traceback (most recent call last):
  File "/home/cyrus/miniconda3/envs/dev-py311/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 770, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 


KeyboardInterrupt: 

In [ ]:
Hmn

In [14]:
Hmn.subs({m: 1, n: 1}).as_expr()

1.0*Integral(1.0*Wavefunction(0.316227766016838*sin(0.05*pi*x), x)*(0.25*x**2*Wavefunction(0.316227766016838*sin(0.05*pi*x), x) - 5.0*x*Wavefunction(0.316227766016838*sin(0.05*pi*x), x) - 1.0*Wavefunction(-0.000790569415042095*pi**2*sin(0.05*pi*x), x) + 25.0*Wavefunction(0.316227766016838*sin(0.05*pi*x), x)), (x, 0, 20.0))

In [109]:
hmn

(0.00025*pi**2*n**2 + 2.5*(0.1*x - 1)**2)*sin(0.05*pi*m*x)*sin(0.05*pi*n*x)

In [117]:
from sympy import N
N(integrate(hmn.evalf(subs={m: 1, n: 1}), (x, -3*L, 4*L)))

2823.04363713554

In [8]:
# Construct the matrix
matrix_elements = []
for i in range(nbasis):
    row = []
    for j in range(nbasis):
        # Substitute m and n values into Hmn and evaluate
        hmn.evalf(subs={m: i, n: j})
        element = Hmn.subs({m: i, n: j}).doit()
        row.append(element)
    matrix_elements.append(row)

# Print the matrix
for row in matrix_elements:
    print(row)

KeyboardInterrupt: 

In [9]:
matrix_elements

[[146.666666666667*Wavefunction(0, 20.0)**2,
  1.0*Integral(1.0*Wavefunction(0, x)*(0.25*x**2*Wavefunction(0.316227766016838*sin(0.05*pi*x), x) - 5.0*x*Wavefunction(0.316227766016838*sin(0.05*pi*x), x) - 1.0*Wavefunction(-0.000790569415042095*pi**2*sin(0.05*pi*x), x) + 25.0*Wavefunction(0.316227766016838*sin(0.05*pi*x), x)), (x, 0, 20.0)),
  1.0*Integral(1.0*Wavefunction(0, x)*(0.25*x**2*Wavefunction(0.316227766016838*sin(0.1*pi*x), x) - 5.0*x*Wavefunction(0.316227766016838*sin(0.1*pi*x), x) - 1.0*Wavefunction(-0.00316227766016838*pi**2*sin(0.1*pi*x), x) + 25.0*Wavefunction(0.316227766016838*sin(0.1*pi*x), x)), (x, 0, 20.0)),
  1.0*Integral(1.0*Wavefunction(0, x)*(0.25*x**2*Wavefunction(0.316227766016838*sin(0.15*pi*x), x) - 5.0*x*Wavefunction(0.316227766016838*sin(0.15*pi*x), x) - 1.0*Wavefunction(-0.00711512473537885*pi**2*sin(0.15*pi*x), x) + 25.0*Wavefunction(0.316227766016838*sin(0.15*pi*x), x)), (x, 0, 20.0)),
  1.0*Integral(1.0*Wavefunction(0, x)*(0.25*x**2*Wavefunction(0.316227

In [32]:
Derivative(f(x), x) == diff(f(x), x)

True

In [ ]:
Derivative

In [33]:
diff(f(x), x, x)

Derivative(f(x), (x, 2))

In [12]:
f = phi.integrate(x)

In [18]:
f.evalf(subs={n:3})

-0.671056161393161*cos(3*pi*x/20)